In [ ]:
import wandb

In [ ]:
!wandb login


In [ ]:
import wandb
import os
os.environ['WANDB_DIR'] = os.getcwd() + '/wandb/'
os.environ['WANDB_CACHE_DIR'] = os.getcwd() + '/wandb/.cache/'
os.environ['WANDB_CONFIG_DIR'] = os.getcwd() + '/wandb/.config/'

import torch
import torch.nn as nn
import torch.optim as optim

# 初始化wandb
wandb.init(
    entity='kaifan-li',
    project="my_project"
)

# 构建模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练循环
for epoch in range(10):
    running_loss = 0.0
    for i in range(100):
        inputs = torch.randn(32, 10)  # 随机生成输入数据
        labels = torch.randn(32, 1)   # 随机生成标签
        
        # 正向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # 记录训练过程和指标
    avg_loss = running_loss / 100
    wandb.log({"epoch": epoch, "loss": avg_loss})


----

In [3]:
from model.modeling_bart import BartPrefixPropForConditionalGeneration
from transformers import BartTokenizerFast, BartConfig, AutoConfig
from config.custom_config import PromptBartConfig
config = AutoConfig.from_pretrained('facebook/bart-base')
custom_config = PromptBartConfig(
    **config.to_dict(),
)
model = BartPrefixPropForConditionalGeneration.from_pretrained('facebook/bart-base', config=custom_config)
model

Some weights of BartPrefixPropForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['propagated_prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartPrefixPropForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((7

In [4]:
from transformers import AutoTokenizer
text = 'i love you'
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')
input_ids = tokenizer(text, return_tensors='pt')['input_ids']
model.generate(
    input_ids,
    test_flag='this is a test'
)

test_flag='this is a test'
self.config.pre_seq_len=20
this is a flag


/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'shape'